In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json

In [5]:
# Set the search path for files (assuming the directory is relative to the current script)
file_path_mc124 = os.path.join("..", "..","fine_dust_complete", ".csv")
df = pd.read_csv('../data/df_fine_dust_wind_merged.csv', parse_dates=['datetime'])

# rename the wrongly named column -- NEED TO CLEAN THAT UP LATER
df = df.rename(columns={'pm10_value': 'value'})

df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309715 entries, 172992 to 542554
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   datetime        309715 non-null  datetime64[ns]
 1   station         309715 non-null  object        
 2   core            309715 non-null  object        
 3   value           309715 non-null  float64       
 4   hour            309715 non-null  float64       
 5   day             309715 non-null  float64       
 6   month           309715 non-null  float64       
 7   year            309715 non-null  float64       
 8   day_of_week     309715 non-null  float64       
 9   is_weekend      309715 non-null  float64       
 10  wind_speed      309715 non-null  float64       
 11  wind_direction  309715 non-null  float64       
dtypes: datetime64[ns](1), float64(9), object(2)
memory usage: 30.7+ MB


In [6]:
df.sample(20)

,datetime,station,core,value,hour,day,month,year,day_of_week,is_weekend,wind_speed,wind_direction
217652,2016-12-01 11:00:00,mc124,pm10,19.0,11.0,1.0,12.0,2016.0,3.0,0.0,17.7,280.0
258677,2017-12-24 11:00:00,mc124,no2,19.0,11.0,24.0,12.0,2017.0,6.0,1.0,16.3,260.0
451811,2022-05-26 19:00:00,mc124,pm2,8.0,19.0,26.0,5.0,2022.0,3.0,0.0,10.6,260.0
381964,2020-10-31 23:00:00,mc124,nox,16.0,23.0,31.0,10.0,2020.0,5.0,1.0,11.5,180.0
513896,2023-10-27 09:00:00,mc124,pm2,15.0,9.0,27.0,10.0,2023.0,4.0,0.0,6.4,90.0
409734,2021-05-12 12:00:00,mc124,nox,149.0,12.0,12.0,5.0,2021.0,2.0,0.0,5.7,300.0
374946,2020-08-29 10:00:00,mc124,pm2,11.0,10.0,29.0,8.0,2020.0,5.0,1.0,3.7,240.0
256322,2017-11-13 02:00:00,mc124,no2,24.0,2.0,13.0,11.0,2017.0,0.0,0.0,8.0,310.0
280054,2018-06-27 07:00:00,mc124,nox,81.0,7.0,27.0,6.0,2018.0,2.0,0.0,6.6,60.0
391075,2020-12-17 00:00:00,mc124,pm10,27.0,0.0,17.0,12.0,2020.0,3.0,0.0,13.8,160.0


In [12]:
df_prev_wind = pd.read_csv('../data/df_fine_dust_wd_h-1.csv', parse_dates=['datetime'])

df['datetime'] = pd.to_datetime(df['datetime'])
df_prev_wind['datetime'] = pd.to_datetime(df_prev_wind['datetime'])

# Step 4: Merge the dataframes on datetime
df_merged = pd.merge(df, df_prev_wind, on='datetime', how='left')

In [13]:
df_merged.head(20)

,datetime,station_x,core_x,value,hour_x,day_x,month_x,year_x,day_of_week_x,is_weekend_x,...,hour_y,day_y,month_y,year_y,day_of_week_y,is_weekend_y,wind_speed_y,wind_direction_y,wind_speed_h-1,wind_direction_h-1
0,2015-08-31 23:00:00,mc124,no2,116.0,23.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-08-31 23:00:00,mc124,no,81.0,23.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-08-31 23:00:00,mc124,nox,240.0,23.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-08-31 22:00:00,mc124,no2,133.0,22.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-08-31 22:00:00,mc124,no,92.0,22.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-08-31 22:00:00,mc124,nox,274.0,22.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-08-31 21:00:00,mc124,no2,136.0,21.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-08-31 21:00:00,mc124,no,68.0,21.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-08-31 21:00:00,mc124,nox,239.0,21.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-08-31 20:00:00,mc124,no2,121.0,20.0,31.0,8.0,2015.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
'